Considering Adding:
- Confidence intervals for performance metrics (Most papers only really seem to go this far)
- Statistical significance tests between different approaches
- Variance analysis across multiple runs




In [1]:
!pip install optuna

In [2]:
import sys
print(sys.executable)


import optuna
print(optuna.__version__)

/home/sd37/.conda/envs/thesis/bin/python
4.1.0


/home/sd37/.conda/envs/thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **This experiment is investigating the performance of an adaptive reward function to state of the art reward functions in environments with environmentally variable changes.**

-> Is there a statisitcally significant improvement in performance over time in this varying environment.


In [3]:
import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
import random
import torch

import sys
from pathlib import Path


current_dir = os.getcwd()  
project_root = str(Path(current_dir).parent.parent)
sys.path.append(project_root)


# Initialize environment and device
from AdaptiveRewardFunctionLearning.Prompts.prompts import device, apiKey,modelName

#Cu stomCartPoleEnv
from RLEnvironment.env import CustomCartPoleEnv
#RewardUpdateSystem
from AdaptiveRewardFunctionLearning.RewardGeneration.rewardCritic import RewardUpdateSystem
#DQLearningAgent
from RLEnvironment.training.agent import DQLearningAgent
from RLEnvironment.training.training import trainDQLearning

#DynamicRewardFunction
from AdaptiveRewardFunctionLearning.RewardGeneration.rewardCodeGeneration import stabilityReward, efficiencyReward, dynamicRewardFunction

#import
from AdaptiveRewardFunctionLearning.Visualisation.trainingTestFunctions import (
    runEpisode,
    detectJumps,
    analyzeRewardSensibility,
    performUpdate,
    updateCompositeRewardFunction,
    plotExperimentResults,
    savePlot
)

# Import new reward functions
from AdaptiveRewardFunctionLearning.RewardGeneration.cartpole_energy_reward import EnergyBasedRewardFunction


# from AdaptiveRewardFunctionLearning.RewardGeneration.cartpole_meta_learning import meta_learning_cartpole
# from AdaptiveRewardFunctionLearning.RewardGeneration.reward_meta_learning import RewardFunctionMetaLearner

**State of the Art Reward Functions with Reference to Papers**

**Potential-based Reward Shaping (PBRS):**
```python
def potentialBasedRewardShaping(observation, action):
    x, xDot, angle, angleDot = observation
    gamma = 0.99  # Example discount factor

    def phi(x, xDot, angle, angleDot):
        # Example potential function
        return -abs(x) - abs(angle)

    current_potential = phi(x, xDot, angle, angleDot)
    next_potential = phi(x + xDot, angle + angleDot, xDot, angleDot)  # Simplified next state
    return float(gamma * next_potential - current_potential)
```

Paper: "Potential-based Shaping in Model-based Reinforcement Learning"

Link: https://cdn.aaai.org/AAAI/2008/AAAI08-096.pdf


**Parameterized Reward Shaping:**
```python
def parameterizedRewardShaping(observation, action):
    x, xDot, angle, angleDot = observation
    original_reward = 1.0  # Assuming default CartPole reward

    def f(x, xDot, angle, angleDot):
        # Example shaping reward function
        return -abs(angle)

    def z_phi(x, xDot, angle, angleDot):
        # Example shaping weight function
        return 0.5

    shaping_reward = f(x, xDot, angle, angleDot)
    shaping_weight = z_phi(x, xDot, angle, angleDot)
    return float(original_reward + shaping_weight * shaping_reward)
```

Paper: "Learning to Utilize Shaping Rewards: A New Approach of Reward Shaping"

Link: http://arxiv.org/pdf/2011.02669.pdf


**Energy Based Reward Function - Physics Based**

```python
def energyBasedReward(observation, action):
    x, xDot, angle, angleDot = observation
    
    # Calculate kinetic and potential energy components
    kineticEnergy = 0.5 * (xDot**2 + angleDot**2)
    potentialEnergy = 9.8 * (1 + cos(angle))  # g * h, where h depends on angle
    
    # Reward is inverse of total energy (less energy = more stable = better reward)
    energyPenalty = -(kineticEnergy + potentialEnergy)
    return float(1.0 + 0.1 * energyPenalty)  # Base reward plus energy term
```

Paper: "Energy-Based Control for Safe Robot Learning" (2019)

Link: https://ieeexplore.ieee.org/document/8794207


**Baseline Reward Function:**
```python
def baselineCartPoleReward(observation, action):
    return 1.0
```

### **Performance Experiment**

In [4]:

# Initialize reward functions and meta-learners
energy_reward = EnergyBasedRewardFunction(mass_cart=1.0, mass_pole=0.1, length=0.5, gravity=9.8)
# meta_reward = RewardFunctionMetaLearner(state_dim=4, action_dim=1)  # CartPole has 4 state dims, 1 action dim

# def potentialBasedRewardShaping(observation, action):
#     """Advanced potential-based reward shaping using meta-learning"""
#     reward_func = meta_reward.generate_reward_function()
#     return float(reward_func(observation, action))

# def parameterizedRewardShaping(observation, action):
#     """Meta-learning based parameterized reward shaping"""
#     # Use meta-learning framework for parameter optimization
#     learner = meta_learning_cartpole()
#     return float(learner.parameterized_reward(observation, action))

def energyBasedReward(observation, action):
    """Enhanced physics-based energy reward"""
    return float(energy_reward.compute_reward(observation, action))


def potentialBasedReward(observation, action):
    """Potential-based reward shaping for CartPole  - This one is not dynamic"""
    x, x_dot, theta, theta_dot = observation
    gamma = 0.99
    
    def potential(state):
        # Potential function based on cart position and pole angle
        # Higher potential for centered cart and upright pole
        cart_potential = -(state[0] ** 2)  # Penalize distance from center
        angle_potential = -((state[2] ** 2))  # Penalize angle from vertical
        velocity_potential = -(state[1] ** 2)  # Penalize high velocities
        ang_velocity_potential = -(state[3] ** 2)  # Penalize high angular velocities
        
        return cart_potential + 2*angle_potential + velocity_potential + ang_velocity_potential

    current_potential = potential(observation)
    next_potential = potential([x + x_dot, x_dot, theta + theta_dot, theta_dot])
    
    # PBRS formula: γΦ(s') - Φ(s)
    shaped_reward = gamma * next_potential - current_potential
    
    return 1.0 + shaped_reward


def baselineReward(observation, action):
    """Standard baseline reward"""
    return 1.0

In [5]:
def runPerformanceComparisonTest(
    episodes=1000, 
    changeInterval=500, 
    lengthchanges=[0.5, 1.5],
    mass_cart=1.0,
    mass_pole=0.1,
    initial_length=0.5,
    gravity=9.8,
    seed=42
):
    print(f"Starting Performance Comparison Test with seed {seed}...")
    
    # Set all random seeds
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    def create_fresh_env():
        # Helper function to create fresh environment with consistent settings
        env = gym.make('CartPole-v1', max_episode_steps=20000, render_mode=None)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        env.reset(seed=seed)
        env = CustomCartPoleEnv(env, numComponents=2)
        env.setEnvironmentParameters(masscart=mass_cart, length=lengthchanges[0], gravity=gravity)
        return env
    
    def create_dqn_agent(env, device):
        return DQLearningAgent(
            env=env, 
            stateSize=4, 
            actionSize=2, 
            device=device,
            learningRate=0.0005,  # Reduced from 0.001
            discountFactor=0.99,
            epsilon=1.0,
            epsilonDecay=0.999,  # Slower decay
            epsilonMin=0.01,
            replayBufferSize=100000,
            batchSize=32,  # Back to original
            targetUpdateFreq=200  # Much less frequent updates
        )
    
    # Initialize energy-based reward function
    energy_reward = EnergyBasedRewardFunction(
        mass_cart=mass_cart, 
        mass_pole=mass_pole, 
        length=initial_length, 
        gravity=gravity
    )
    
    # Create initial environment
    env = create_fresh_env()
    
    # Define reward function configurations
    rewardfunctions = {
        'adaptivereward': {
            'agent': None,  # Will be created fresh for each test
            'updatesystem': RewardUpdateSystem(apiKey, modelName),
            'rewardfunction': None,
            'update_method': 'llm'
        },
        'pbrs': {
            'agent': None,  # Will be created fresh for each test
            'updatesystem': None,
            'rewardfunction': potentialBasedReward,
            'update_method': None
        },
        'energy_based': {
            'agent': None,  # Will be created fresh for each test
            'updatesystem': energy_reward,
            'rewardfunction': energy_reward.compute_reward,
            'update_method': 'physics'
        },
        'baseline': {
            'agent': None,  # Will be created fresh for each test
            'updatesystem': None,
            'rewardfunction': baselineReward,
            'update_method': None
        }
    }

    results = {}
    test_order = ['adaptivereward', 'energy_based', 'baseline', 'pbrs']
    
    # Test each reward function
    for rewardname in test_order:
        print(f"\nTesting reward function: {rewardname}")
        
        # Create fresh environment and agent for each test
        env = create_fresh_env()
        rewardfunctions[rewardname]['agent'] = create_dqn_agent(env, device)
        rewardinfo = rewardfunctions[rewardname]
        
        # Reset length index for each test
        currentlengthidx = 0
        
        if rewardname == 'adaptivereward':
            # Initialize both components for adaptive reward
            env.setComponentReward(1, stabilityReward)
            env.setComponentReward(2, efficiencyReward)
            rewardinfo['updatesystem'].lastUpdateEpisode = 0
        else:
            env.setRewardFunction(rewardinfo['rewardfunction'])
        
        episoderewards = []
        episodebalancetimes = []
        rewardchangeepisodes = []
        
        def onEpisodeEnd(env, updatesystem, episode, reward, steps):
            nonlocal episoderewards, episodebalancetimes, rewardchangeepisodes, currentlengthidx
            
            # Record episode results
            episoderewards.append(reward)
            episodebalancetimes.append(steps)
            
            # Create metrics dictionary
            metrics = {
                'currentEpisode': episode,
                'recentRewards': episoderewards[-100:] if len(episoderewards) > 100 else episoderewards,
                'averageBalanceTime': np.mean(episodebalancetimes[-100:]) if episodebalancetimes else 0,
                'balanceTimeVariance': np.var(episodebalancetimes[-100:]) if len(episodebalancetimes) > 1 else 0
            }
            
            # Debug print for metrics
            if episode % 1000 == 0:
                print(f"\nMetrics Debug at Episode {episode}:")
                print(f"Recent Average Reward: {np.mean(metrics['recentRewards']):.2f}")
                print(f"Average Balance Time: {metrics['averageBalanceTime']:.2f}")
                print(f"Balance Time Variance: {metrics['balanceTimeVariance']:.2f}")
                
                if rewardname == 'adaptivereward' and hasattr(env, 'getCurrentWeights'):
                    weights = env.getCurrentWeights()
                    print(f"Component Weights - Stability: {weights['stability']:.2f}, "
                          f"Efficiency: {weights['efficiency']:.2f}")
            
            # Handle LLM updates only for adaptive reward
            if rewardname == 'adaptivereward' and updatesystem is not None:
                if episode % 1000 == 0:
                    print(f"\nEpisode {episode} - Time Since Last Update: {episode - updatesystem.lastUpdateEpisode}")
                
                for component in range(1, 3):
                    updatesystem.targetComponent = component
                    if updatesystem.waitingTime(f'component_{component}', metrics, updatesystem.lastUpdateEpisode):
                        current_func = env.rewardComponents[f'rewardFunction{component}']
                        new_function, updated = updatesystem.validateAndUpdate(current_func)
                        
                        if updated:
                            env.setComponentReward(component, new_function)
                            rewardchangeepisodes.append(episode)
                            updatesystem.lastUpdateEpisode = episode
                            print(f"✓ LLM update for component {component} at episode {episode}")
            
            # Handle physics-based updates
            elif rewardinfo['update_method'] == 'physics':
                if episode % changeInterval == 0 and episode > 0:
                    print(f"\nUpdating physics-based reward at episode {episode}")
                    updatesystem.length = lengthchanges[currentlengthidx]
                    env.setRewardFunction(updatesystem.compute_reward)
                    rewardchangeepisodes.append(episode)
                    print("✓ Physics-based update completed")
            
            # Environment changes
            if episode % changeInterval == 0 and episode > 0:
                currentlengthidx = (currentlengthidx + 1) % len(lengthchanges)
                newlength = lengthchanges[currentlengthidx]
                env.setEnvironmentParameters(length=newlength)
                print(f"\nChanged pole length to: {newlength}m at episode {episode}")
        
        # Train the agent
        agent, env, rewards = trainDQLearning(
            agent=rewardinfo['agent'],
            env=env,
            numEpisodes=episodes,
            updateSystem=rewardinfo['updatesystem'],
            onEpisodeEnd=onEpisodeEnd
        )
        
        # Store results
        results[rewardname] = {
            'rewards': episoderewards,
            'balancetimes': episodebalancetimes,
            'rewardChanges': rewardchangeepisodes
        }
        
        # Print final performance metrics
        print(f"\nCompleted testing {rewardname}")
        print(f"Final average reward: {np.mean(episoderewards[-100:]):.2f}")
        print(f"Final average balance time: {np.mean(episodebalancetimes[-100:]):.2f}")
    
    return results

In [6]:
# Run Experiment

changeInterval = 20000

# results = runPerformanceComparisonTest(
#     episodes=40000,  
#     changeInterval=changeInterval,
#     mass_cart=1.0,
#     lengthchanges=[0.3, 0.9]  
# )

In [7]:
def visualizePerformanceComparison(results, changeInterval, folder_path):
    """
    Create and save performance comparison visualizations
    """
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))
    
    # Color map for different reward functions
    colors = ['b', 'g', 'r', 'c', 'm']
    
    # Plot rewards for each reward function with variance
    for idx, (rewardname, rewardresults) in enumerate(results.items()):
        rewards = pd.Series(rewardresults['rewards'])
        
        # Calculate rolling mean and standard deviation for rewards
        window = 50
        rolling_mean_rewards = rewards.rolling(window=window).mean()
        rolling_std_rewards = rewards.rolling(window=window).std()
        
        # Plot mean line for rewards
        ax1.plot(rolling_mean_rewards, 
                label=f'{rewardname}', 
                linewidth=2, 
                color=colors[idx])
        
        # Plot variance area for rewards
        ax1.fill_between(
            range(len(rewards)),
            rolling_mean_rewards - rolling_std_rewards,
            rolling_mean_rewards + rolling_std_rewards,
            color=colors[idx],
            alpha=0.2
        )
        
        # Add vertical lines for environment changes (red)
        change_episodes = range(changeInterval, len(rewards), changeInterval)
        for ep in change_episodes:
            ax1.axvline(x=ep, color='r', linestyle='--', alpha=0.3,
                       label='Environment Change' if ep == change_episodes[0] else None)
        
        # Add vertical lines for reward function changes (green)
        if 'rewardChanges' in rewardresults:
            for ep in rewardresults['rewardChanges']:
                ax1.axvline(x=ep, color='g', linestyle='--', alpha=0.3,
                          label=f'{rewardname} Update' if ep == rewardresults['rewardChanges'][0] else None)
    
    ax1.set_title('Average Reward Over Time with Variance')
    ax1.set_xlabel('Episode')
    ax1.set_ylabel('Reward')
    ax1.legend()
    ax1.grid(True)
    
    # Plot balance times
    for idx, (rewardname, rewardresults) in enumerate(results.items()):
        balancetimes = pd.Series(rewardresults['balancetimes'])
        
        rolling_mean_balance = balancetimes.rolling(window=window).mean()
        rolling_std_balance = balancetimes.rolling(window=window).std()
        
        ax2.plot(rolling_mean_balance,
                label=f'{rewardname}', 
                linewidth=2, 
                color=colors[idx])
        
        ax2.fill_between(
            range(len(balancetimes)),
            rolling_mean_balance - rolling_std_balance,
            rolling_mean_balance + rolling_std_balance,
            color=colors[idx],
            alpha=0.2
        )
        
        # Add vertical lines for environment changes
        for ep in change_episodes:
            ax2.axvline(x=ep, color='r', linestyle='--', alpha=0.3,
                       label='Environment Change' if ep == change_episodes[0] else None)
        
        # Add vertical lines for reward function changes
        if 'rewardChanges' in rewardresults:
            for ep in rewardresults['rewardChanges']:
                ax2.axvline(x=ep, color='g', linestyle='--', alpha=0.3,
                          label=f'{rewardname} Update' if ep == rewardresults['rewardChanges'][0] else None)
    
    ax2.set_title('Average Balance Time Over Episodes with Variance')
    ax2.set_xlabel('Episode')
    ax2.set_ylabel('Steps')
    ax2.legend()
    ax2.grid(True)
    
    # Plot environment parameters
    env_param_history = []
    for episode in range(len(next(iter(results.values()))['rewards'])):
        idx = (episode // changeInterval) % 2
        length = 0.3 if idx == 0 else 0.9  # Alternating between 0.3 and 0.9
        env_param_history.append(length)
    
    ax3.plot(env_param_history, label='Pole Length', color='purple')
    ax3.set_title('Environment Parameters Over Episodes')
    ax3.set_xlabel('Episode')
    ax3.set_ylabel('Pole Length (m)')
    ax3.grid(True)
    ax3.legend()
    
    plt.tight_layout()
    
    # Save plots
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(folder_path, f"performance_comparison_{timestamp}.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved plot: performance_comparison_{timestamp}.png in {folder_path}")
    plt.close()
    
    
    
def calculateStability(rewards):
    """
    Calculate stability score based on reward variance in the last 100 episodes
    Lower variance = higher stability
    """
    if len(rewards) < 100:
        return 0.0
    
    last_hundred = rewards[-100:]
    mean_reward = np.mean(last_hundred)
    if mean_reward == 0:
        return 0.0
        
    # Calculate coefficient of variation (normalized standard deviation)
    stability = 1 - (np.std(last_hundred) / mean_reward)
    return max(0, min(1, stability))  # Normalize between 0 and 1

def calculateConvergenceTime(rewards, threshold=195, window=50):
    """
    Calculate the number of episodes needed to reach and maintain a certain performance
    threshold for a given window of episodes
    """
    if len(rewards) < window:
        return len(rewards)
    
    rolling_mean = pd.Series(rewards).rolling(window).mean()
    
    for episode in range(window, len(rewards)):
        if rolling_mean[episode] >= threshold:
            # Check if performance is maintained
            maintained = all(avg >= threshold * 0.9 for avg in rolling_mean[episode:episode+window])
            if maintained:
                return episode
    
    return len(rewards)  # If never converged, return total episodes

def calculatePerformanceMetrics(results):
    """Calculate performance metrics for each reward type"""
    metrics = {}
    for rewardname, rewardresults in results.items():
        metrics[rewardname] = {
            'finalavgreward': np.mean(rewardresults['rewards'][-100:]),
            'finalavgbalance': np.mean(rewardresults['balancetimes'][-100:]),
            'convergencetime': calculateConvergenceTime(rewardresults['rewards']),
            'stability': calculateStability(rewardresults['rewards'])
        }
    return pd.DataFrame(metrics).T
    
    
    
    
    
def saveMetricsTable(metrics, filename, folder_path):
    """Save metrics table to specified folder"""
    # Create figure for metrics table
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.axis('tight')
    ax.axis('off')
    
    # Create table with formatted metrics
    table = ax.table(
        cellText=metrics.values.round(3),
        colLabels=metrics.columns,
        rowLabels=metrics.index,
        cellLoc='center',
        loc='center'
    )
    
    # Adjust font size and scaling
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.2, 1.5)
    
    plt.title("Performance Metrics Comparison")
    
    # Save with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(folder_path, f"{filename}_{timestamp}.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved metrics table: {filename}_{timestamp}.png in {folder_path}")
    plt.close()



In [8]:

# Visualize the results
# visualizePerformanceComparison(results,changeInterval)


# # Calculate and display the metrics
# metrics = calculatePerformanceMetrics(results)
# print("\nPerformance Metrics:")
# print(metrics)
    
# Call the function
# saveMetricsTable(metrics)

### Multiple runs to generate confidence intervals

In [9]:
def createExperimentFolder():
    """Create a timestamped folder for experiment results"""
    from datetime import datetime
    import os
    
    # Create base experiment folder if it doesn't exist
    if not os.path.exists("PerformanceExperiment"):
        os.makedirs("PerformanceExperiment")
    
    # Create timestamped subfolder
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    experiment_folder = os.path.join("PerformanceExperiment", f"experiment_{timestamp}")
    os.makedirs(experiment_folder)
    
    return experiment_folder

def savePlot(fig, filename, folder_path):
    """Save plot to specified folder with timestamp"""
    from datetime import datetime
    import os
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(folder_path, f"{filename}_{timestamp}.png")
    fig.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved plot: {filename}_{timestamp}.png in {folder_path}")
    
def saveMetricsTable(metrics, filename, folder_path):
    """Save metrics table to specified folder"""
    import matplotlib.pyplot as plt
    from datetime import datetime
    import os
    
    # Style the DataFrame for visualization
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=metrics.values,
                    colLabels=metrics.columns,
                    rowLabels=metrics.index,
                    cellLoc='center',
                    loc='center')
    
    # Adjust font size and scaling
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.2, 1.5)
    
    plt.title("Performance Metrics Comparison")
    
    # Save with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join(folder_path, f"{filename}_{timestamp}.png")
    plt.savefig(filepath, bbox_inches='tight', dpi=300)
    print(f"Saved metrics table: {filename}_{timestamp}.png in {folder_path}")
    plt.close()

def runMultipleExperiments(numRuns=4, episodes=40000, changeInterval=20000):
    """
    Run multiple experiments and save results in organized folders
    """
    # Create main experiment folder
    experiment_folder = createExperimentFolder()
    
    allResults = []
    allMetrics = []
    aggregatedMetrics = {}
    
    # Create run folders
    for run in range(numRuns):
        print(f"\nStarting Run {run + 1}/{numRuns}")
        
        # Create folder for this run
        run_folder = os.path.join(experiment_folder, f"run_{run + 1}")
        os.makedirs(run_folder)
        
        # Run experiment
        results = runPerformanceComparisonTest(
            episodes=episodes,
            changeInterval=changeInterval,
            mass_cart=1.0,
            lengthchanges=[0.3, 0.9]
        )
        
        # Calculate metrics for this run
        metrics = calculatePerformanceMetrics(results)
        
        # Store results
        allResults.append(results)
        allMetrics.append(metrics)
        
        # Store metrics by reward type
        for idx, row in metrics.iterrows():
            if idx not in aggregatedMetrics:
                aggregatedMetrics[idx] = []
            aggregatedMetrics[idx].append(row.to_dict())
        
        # Visualize and save individual run results
        visualizePerformanceComparison(results, changeInterval, run_folder)
        saveMetricsTable(metrics, f"metrics_run_{run + 1}", run_folder)
    
    # Calculate confidence intervals (95%)
    confidenceIntervals = {}
    resultsTable = pd.DataFrame()
    
    for rewardType, metrics_list in aggregatedMetrics.items():
        metrics_df = pd.DataFrame(metrics_list)
        means = metrics_df.mean()
        cis = 1.96 * metrics_df.std() / np.sqrt(numRuns)
        
        # Create row for this reward type
        resultsTable.loc[rewardType, 'Average Reward'] = f"{means['finalavgreward']:.2f} ± {cis['finalavgreward']:.2f}"
        resultsTable.loc[rewardType, 'Average Balance'] = f"{means['finalavgbalance']:.2f} ± {cis['finalavgbalance']:.2f}"
        resultsTable.loc[rewardType, 'Convergence Time'] = f"{means['convergencetime']:.2f} ± {cis['convergencetime']:.2f}"
        resultsTable.loc[rewardType, 'Stability'] = f"{means['stability']:.2f} ± {cis['stability']:.2f}"
        
        # Store the raw values for possible further analysis
        confidenceIntervals[rewardType] = {
            'finalavgreward': {'mean': means['finalavgreward'], 'ci': cis['finalavgreward']},
            'finalavgbalance': {'mean': means['finalavgbalance'], 'ci': cis['finalavgbalance']},
            'convergencetime': {'mean': means['convergencetime'], 'ci': cis['convergencetime']},
            'stability': {'mean': means['stability'], 'ci': cis['stability']}
        }
    
    # Save aggregate statistics
    with open(os.path.join(experiment_folder, "aggregate_statistics.txt"), "w") as f:
        f.write("Aggregate Statistics:\n")
        for rewardType, metrics in confidenceIntervals.items():
            f.write(f"\n{rewardType}:\n")
            for metric, values in metrics.items():
                f.write(f"{metric}: {values['mean']:.2f} ± {values['ci']:.2f}\n")
    
    # Save final results table
    saveMetricsTable(resultsTable, "final_results", experiment_folder)
    
    print(f"\nExperiment results saved in: {experiment_folder}")
    return confidenceIntervals, allResults, resultsTable

In [ ]:
# confidenceIntervals, allResults, resultsTable = runMultipleExperiments(
#     numRuns=4,
#     episodes=10000,
#     changeInterval=5000
# )

confidenceIntervals, allResults, resultsTable = runMultipleExperiments(
    numRuns=1,
    episodes=20000,
    changeInterval=10000
)


Maintaining good performance for component_1: 1937.58 vs best 1999.70

Maintaining good performance for component_2: 1937.58 vs best 1999.70

Maintaining good performance for component_1: 1937.92 vs best 1999.20

Maintaining good performance for component_2: 1937.92 vs best 1999.20

Maintaining good performance for component_1: 1937.89 vs best 1999.25

Maintaining good performance for component_2: 1937.89 vs best 1999.25

Maintaining good performance for component_1: 1937.83 vs best 1999.13

Maintaining good performance for component_2: 1937.83 vs best 1999.13

Maintaining good performance for component_1: 1937.62 vs best 1999.19

Maintaining good performance for component_2: 1937.62 vs best 1999.19

Maintaining good performance for component_1: 1937.88 vs best 1999.27

Maintaining good performance for component_2: 1937.88 vs best 1999.27

Maintaining good performance for component_1: 1937.79 vs best 1999.21

Maintaining good performance for component_2: 1937.79 vs best 1999.21

Maint


Maintaining good performance for component_1: 1999.24 vs best 1937.88

Maintaining good performance for component_2: 1999.24 vs best 1937.88

Maintaining good performance for component_1: 1999.37 vs best 1937.64

Maintaining good performance for component_2: 1999.37 vs best 1937.64

Maintaining good performance for component_1: 1999.31 vs best 1937.85

Maintaining good performance for component_2: 1999.31 vs best 1937.85

Maintaining good performance for component_1: 1999.37 vs best 1937.86

Maintaining good performance for component_2: 1999.37 vs best 1937.86

Maintaining good performance for component_1: 1999.38 vs best 1937.86

Maintaining good performance for component_2: 1999.38 vs best 1937.86

Maintaining good performance for component_1: 1999.06 vs best 1937.85

Maintaining good performance for component_2: 1999.06 vs best 1937.85

Maintaining good performance for component_1: 1999.56 vs best 1968.07

Maintaining good performance for component_2: 1999.56 vs best 1968.07

Maint


Maintaining good performance for component_1: 1997.23 vs best 1999.44

Maintaining good performance for component_2: 1997.23 vs best 1999.44

Maintaining good performance for component_1: 1997.32 vs best 1999.30

Maintaining good performance for component_2: 1997.32 vs best 1999.30

Maintaining good performance for component_1: 1997.23 vs best 1999.34

Maintaining good performance for component_2: 1997.23 vs best 1999.34

Maintaining good performance for component_1: 1997.23 vs best 1999.39

Maintaining good performance for component_2: 1997.23 vs best 1999.39

Maintaining good performance for component_1: 1997.22 vs best 1999.38

Maintaining good performance for component_2: 1997.22 vs best 1999.38

Maintaining good performance for component_1: 1997.42 vs best 1999.21

Maintaining good performance for component_2: 1997.42 vs best 1999.21

Maintaining good performance for component_1: 1997.34 vs best 1999.45

Maintaining good performance for component_2: 1997.34 vs best 1999.45

Maint


Maintaining good performance for component_1: 1961.16 vs best 1997.23

Maintaining good performance for component_2: 1961.16 vs best 1997.23

Maintaining good performance for component_1: 1961.29 vs best 1997.20

Maintaining good performance for component_2: 1961.29 vs best 1997.20

Maintaining good performance for component_1: 1961.29 vs best 1997.20

Maintaining good performance for component_2: 1961.29 vs best 1997.20

Maintaining good performance for component_1: 1961.22 vs best 1997.25

Maintaining good performance for component_2: 1961.22 vs best 1997.25

Maintaining good performance for component_1: 1961.03 vs best 1997.39

Maintaining good performance for component_2: 1961.03 vs best 1997.39

Maintaining good performance for component_1: 1961.19 vs best 1997.23

Maintaining good performance for component_2: 1961.19 vs best 1997.23

Maintaining good performance for component_1: 1961.21 vs best 1997.26

Maintaining good performance for component_2: 1961.21 vs best 1997.26

Maint


Maintaining good performance for component_1: 1997.85 vs best 1961.17

Maintaining good performance for component_2: 1997.85 vs best 1961.17

Maintaining good performance for component_1: 1997.76 vs best 1961.24

Maintaining good performance for component_2: 1997.76 vs best 1961.24

Maintaining good performance for component_1: 1994.21 vs best 1961.38

Maintaining good performance for component_2: 1994.21 vs best 1961.38

Maintaining good performance for component_1: 1994.70 vs best 1961.27

Maintaining good performance for component_2: 1994.70 vs best 1961.27

Maintaining good performance for component_1: 1994.80 vs best 1967.63

Maintaining good performance for component_2: 1994.80 vs best 1967.63

Maintaining good performance for component_1: 1994.63 vs best 2000.22

Maintaining good performance for component_2: 1994.63 vs best 2000.22

Maintaining good performance for component_1: 1994.59 vs best 1999.92

Maintaining good performance for component_2: 1994.59 vs best 1999.92

Maint

...


Maintaining good performance for component_1: 278.92 vs best 346.08

Maintaining good performance for component_2: 278.92 vs best 346.08

Maintaining good performance for component_1: 276.72 vs best 338.15

Maintaining good performance for component_2: 276.72 vs best 338.15

Maintaining good performance for component_1: 286.81 vs best 318.93

Maintaining good performance for component_2: 286.81 vs best 318.93

Maintaining good performance for component_1: 284.50 vs best 317.13

Maintaining good performance for component_2: 284.50 vs best 317.13

Maintaining good performance for component_1: 278.39 vs best 322.28

Maintaining good performance for component_2: 278.39 vs best 322.28

Maintaining good performance for component_1: 278.03 vs best 313.89

Maintaining good performance for component_2: 278.03 vs best 313.89

Maintaining good performance for component_1: 277.28 vs best 314.72

Maintaining good performance for component_2: 277.28 vs best 314.72

Maintaining good performance for 


Maintaining good performance for component_1: 254.06 vs best 285.48

Maintaining good performance for component_2: 254.06 vs best 285.48

Maintaining good performance for component_1: 251.19 vs best 298.94

Maintaining good performance for component_2: 251.19 vs best 298.94

Maintaining good performance for component_1: 253.75 vs best 303.76

Maintaining good performance for component_2: 253.75 vs best 303.76

Maintaining good performance for component_1: 254.00 vs best 305.29

Maintaining good performance for component_2: 254.00 vs best 305.29

Maintaining good performance for component_1: 251.95 vs best 307.96

Maintaining good performance for component_2: 251.95 vs best 307.96

Maintaining good performance for component_1: 252.67 vs best 309.64

Maintaining good performance for component_2: 252.67 vs best 309.64

Maintaining good performance for component_1: 255.44 vs best 303.50

Maintaining good performance for component_2: 255.44 vs best 303.50

Maintaining good performance for 


Maintaining good performance for component_1: 315.16 vs best 255.78

Maintaining good performance for component_2: 315.16 vs best 255.78

Maintaining good performance for component_1: 312.76 vs best 257.50

Maintaining good performance for component_2: 312.76 vs best 257.50

Maintaining good performance for component_1: 308.37 vs best 260.12

Maintaining good performance for component_2: 308.37 vs best 260.12

Maintaining good performance for component_1: 307.01 vs best 260.39

Maintaining good performance for component_2: 307.01 vs best 260.39

Maintaining good performance for component_1: 299.60 vs best 266.54

Maintaining good performance for component_2: 299.60 vs best 266.54

Maintaining good performance for component_1: 299.23 vs best 263.27

Maintaining good performance for component_2: 299.23 vs best 263.27

Maintaining good performance for component_1: 314.51 vs best 263.98

Maintaining good performance for component_2: 314.51 vs best 263.98

Maintaining good performance for 


Maintaining good performance for component_1: 258.04 vs best 287.42

Maintaining good performance for component_2: 258.04 vs best 287.42

Maintaining good performance for component_1: 262.25 vs best 287.02

Maintaining good performance for component_2: 262.25 vs best 287.02

Maintaining good performance for component_1: 263.30 vs best 286.20

Maintaining good performance for component_2: 263.30 vs best 286.20

Maintaining good performance for component_1: 281.62 vs best 286.18

Maintaining good performance for component_2: 281.62 vs best 286.18

Maintaining good performance for component_1: 282.50 vs best 285.52

Maintaining good performance for component_2: 282.50 vs best 285.52

Maintaining good performance for component_1: 291.54 vs best 284.76

Maintaining good performance for component_2: 291.54 vs best 284.76

Maintaining good performance for component_1: 278.14 vs best 295.18

Maintaining good performance for component_2: 278.14 vs best 295.18

Maintaining good performance for 


Error during update: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}

Performance Analysis for component_1:
Current Performance: 129.11
Historical Best: 282.93
Relative Performance: 45.6%

Generating new stability reward function...

Proposed Function:
Here's a modified stability reward function with detailed inline comments, adhering to the given constraints and requirements:

```python
def reward_function(observation, action):
    # Extract relevant state variables from observation
    cart_position = observation[0]
    pole_angle = observation[2]
    pole_angular_velocity = observation[3]

    # Component 1: Reward for keeping the pole upright
    angle_reward = 1.0 - abs(pole_angle) / 0.418  # 0.418 radians ≈ 24 degrees
    
    # Component 2: Penalty for cart moving too far from center
    position_penalty = -abs(cart_position) / 2.4  # 2.4 is the track half-length
    
    # Component 3: Small penalty for high angular velocity 